In [53]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from torch import nn
MODEL_NAME = 'bert-base-multilingual-cased'

'bert-base-multilingual-cased'의 경우 BertTokenizerFast 토크나이저 클래스를 활용함

In [2]:
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(type(model))
print(type(tokenizer))

<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [3]:
for idx, i in enumerate(tokenizer.vocab):
    if idx <=7:
        print(i, tokenizer.vocab[i]) # 텍스트, 인코딩ID
    else:
        break
print(tokenizer.vocab_size)

пункты 22593
julkaisi 45839
portail 106318
##ds 13268
samom 61677
stanja 104065
##weld 93423
প্রথম 21716
119547


In [4]:
text = '이순신은 조건 중기의 무신이다.'

In [5]:
tokenized_input_text = tokenizer(text, return_tensors='pt') # 파이토치 텐서로 리턴
for key, value in tokenized_input_text.items():
    print(key, value)

input_ids tensor([[   101,   9638, 119064,  25387,  10892,   9678,  71439,   9694,  46874,
           9294,  25387,  11925,    119,    102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


`input_ids`: 토크나이징 결과 각 토큰의 vocab ID가 담긴 텐서  
`token_type_ids`: 각 토큰이 어떤 문장에 포함되는 지 ID가 담긴 텐서  
`attention_mask`: 패딩 토큰이 포함되어 있을 경우 해당 attention value는 0


In [6]:
tokenized_input_text.input_ids # <=> tokenized_input_text['input_ids']

tensor([[   101,   9638, 119064,  25387,  10892,   9678,  71439,   9694,  46874,
           9294,  25387,  11925,    119,    102]])

`tokenizer.tokenize(text)`: 입력된 텍스트의 토크나이징 결과를 리턴

## Tokenize, Encode, Decode

In [7]:
tokenized_text = tokenizer.tokenize(text)
input_ids = tokenizer.encode(text) # 알아서 스페셜 토큰을 부착한 모습(101-[CLS], 102-[SEP])
decoded_ids = tokenizer.decode(input_ids)
print(f'tokenize: {tokenized_text}')
print(f'encode: {input_ids}')
print(f'decode: {decoded_ids}') # 알아서 스페셜 토큰을 부착한 모습

tokenize: ['이', '##순', '##신', '##은', '조', '##건', '중', '##기의', '무', '##신', '##이다', '.']
encode: [101, 9638, 119064, 25387, 10892, 9678, 71439, 9694, 46874, 9294, 25387, 11925, 119, 102]
decode: [CLS] 이순신은 조건 중기의 무신이다. [SEP]


In [8]:
tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
input_ids = tokenizer.encode(text, add_special_tokens=False)
decoded_ids = tokenizer.decode(input_ids)
print(f'tokenize: {tokenized_text}')
print(f'encode: {input_ids}')
print(f'decode: {decoded_ids}') # 스페셜 토큰을 부착하지 않을 수도 있음

tokenize: ['이', '##순', '##신', '##은', '조', '##건', '중', '##기의', '무', '##신', '##이다', '.']
encode: [9638, 119064, 25387, 10892, 9678, 71439, 9694, 46874, 9294, 25387, 11925, 119]
decode: 이순신은 조건 중기의 무신이다.


In [9]:
tokenized_text = tokenizer.tokenize(
    text,
    add_special_tokens=False,
    max_length=5, # 최대 길이 설정
    truncation=True
    )
print('tokenize:', tokenized_text)

input_ids = tokenizer.encode(
    text,
    add_special_tokens=False,
    max_length=5, # 최대 길이 설정
    truncation=True
    )
print('input_ids', input_ids)
print(tokenizer.decode(input_ids))

tokenize: ['이', '##순', '##신', '##은', '조']
input_ids [9638, 119064, 25387, 10892, 9678]
이순신은 조


## Padding

In [10]:
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)

[PAD]
0


In [11]:
tokenized_text = tokenizer.tokenize(
    text,
    add_special_tokens=False,
    max_length=20, # 최대 길이 설정
    padding='max_length'
    )
print('tokenize', tokenized_text) # 길이에 맞춰 [PAD] 토큰이 추가된 모습

tokenize ['이', '##순', '##신', '##은', '조', '##건', '중', '##기의', '무', '##신', '##이다', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [12]:
input_ids = tokenizer.encode(
    text,
    add_special_tokens=False,
    max_length=20, # 최대 길이 설정
    padding='max_length'
    )
print('input_ids', input_ids) # [PAD] 토큰의 인코딩 ID 0이 추가된 모습
print('decoded', tokenizer.decode(input_ids))

input_ids [9638, 119064, 25387, 10892, 9678, 71439, 9694, 46874, 9294, 25387, 11925, 119, 0, 0, 0, 0, 0, 0, 0, 0]
decoded 이순신은 조건 중기의 무신이다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


## Update New Token

In [13]:
text = '머리카락이 켸쇽 나와서 우리가 청소 다 해야돼요'

tokenized_text = tokenizer.tokenize(text)
print('tokenize', tokenized_text)

input_ids = tokenizer.encode(text)
print('input_ids', input_ids)
print('decoded', tokenizer.decode(input_ids))

tokenize ['머', '##리', '##카', '##락', '##이', '[UNK]', '나', '##와', '##서', '우', '##리가', '청', '##소', '다', '해', '##야', '##돼', '##요']
input_ids [101, 9265, 12692, 24206, 107693, 10739, 100, 8982, 12638, 12424, 9604, 44130, 9751, 22333, 9056, 9960, 21711, 118798, 48549, 102]
decoded [CLS] 머리카락이 [UNK] 나와서 우리가 청소 다 해야돼요 [SEP]


위처럼 '켸쇽'과 같은 vocabulary에 기록되지 않은 토큰은 `[UNK]` 토큰으로 인코딩. 이러한 토큰이 많을 수록 문장은 본연의 의미를 잃어갈 수밖에 없음 => 새로운 토큰 추가의 필요성

In [14]:
added_token_num = tokenizer.add_tokens(['켸쇽'])
print(added_token_num) # vocab에 새롭게 추가된 토큰 수

1


In [15]:
tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print(tokenized_text)

input_ids = tokenizer.encode(text, add_special_tokens=False)
decoded_ids = tokenizer.decode(input_ids)

print('input_ids', input_ids)
print('decoded_ids', decoded_ids) # vocab이 업데이트되어 '켸쇽'의 인코딩이 [UNK]로 되지 않음

['머', '##리', '##카', '##락', '##이', '켸쇽', '나', '##와', '##서', '우', '##리가', '청', '##소', '다', '해', '##야', '##돼', '##요']
input_ids [9265, 12692, 24206, 107693, 10739, 119547, 8982, 12638, 12424, 9604, 44130, 9751, 22333, 9056, 9960, 21711, 118798, 48549]
decoded_ids 머리카락이 켸쇽 나와서 우리가 청소 다 해야돼요


In [16]:
tokenizer.vocab_size

119547

위키트리 데이터만을 가지고 vocabulary가 구성되어 있으니, 도메인에 맞게 vocabulary를 업데이트해줄 필요가 있음!

## Special Tokens

In [17]:
# [JHKO]라는 스페셜 토큰을 추가해보고 싶은데
text = "[JHKO]이순신은 조선 중기의 무신이다[/JHKO]"

tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print('tokenized', tokenized_text) # 일반적인 토큰으로 인식되어 토크나이징이 의도대로 되지 않는다

tokenized ['[', 'J', '##H', '##KO', ']', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '[', '/', 'J', '##H', '##KO', ']']


이럴 땐 `add_special_tokens` 메서드를 통해 스페셜 토큰을 추가

In [18]:
added_token_num += tokenizer.add_special_tokens({'additional_special_tokens': ['[JHKO]', '[/JHKO]']})

In [19]:
tokenized_text = tokenizer.tokenize(text ,add_special_tokens=False)
print('tokenize', tokenized_text)

input_ids = tokenizer.encode(text ,add_special_tokens=False)
decoded_ids = tokenizer.decode(input_ids)
decoded_ids_skipped = tokenizer.decode(input_ids, skip_special_tokens=True)

print('input_ids', input_ids) # [JHKO] 스페셜 토큰이 119548의 ID로 맵핑된 모습
print('decoded_ids', decoded_ids)
print('decoded_ids_skipped', decoded_ids_skipped) # 스페셜 토큰이 제거된 모습

tokenize ['[JHKO]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '[/JHKO]']
input_ids [119548, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119549]
decoded_ids [JHKO] 이순신은 조선 중기의 무신이다 [/JHKO]
decoded_ids_skipped 이순신은 조선 중기의 무신이다


## 토큰이 새롭게 추가됐다면 신경써야할 부분

이미 pretrained 된 모델은 기존 vocab size에 맞춰져 있기 때문에, 토큰을 새로 추가했을 경우 별도의 모델 교정이 필요

In [27]:
single_seg_input = tokenizer('이순신은 조선 중기의 무신이다')

# [CLS] 토큰이 부여된 모습
print('Single segment token, str', tokenizer.convert_ids_to_tokens(single_seg_input['input_ids']))
print('Single segment token, int', single_seg_input['input_ids'])
print('Single segment type', single_seg_input['token_type_ids'])

Single segment token, str ['[CLS]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '[SEP]']
Single segment token, int [101, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 102]
Single segment type [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [28]:
multi_seg_input = tokenizer('이순신은 조선 중기의 무신이다', '그는 임진왜란을 승리로 이끌었다')

# [CLS], [SEP] 토큰이 부여된 모습
print('multi segment token, str', tokenizer.convert_ids_to_tokens(multi_seg_input['input_ids']))
print('multi segment token, int', multi_seg_input['input_ids'])
print('multi segment type', multi_seg_input['token_type_ids'])

multi segment token, str ['[CLS]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '[SEP]', '그는', '임', '##진', '##왜', '##란', '##을', '승', '##리로', '이', '##끌', '##었다', '[SEP]']
multi segment token, int [101, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 102, 17889, 9644, 18623, 119164, 49919, 10622, 9484, 100434, 9638, 118705, 17706, 102]
multi segment type [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## BERT 사용해보기

In [30]:
text = '이순신은 [MASK] 중기의 무신이다'
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)

['이', '##순', '##신', '##은', '[MASK]', '중', '##기의', '무', '##신', '##이다']


In [33]:
from transformers import pipeline

nlp_fill = pipeline('fill-mask', model=MODEL_NAME)
nlp_fill(text)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'sequence': '[CLS] 이순신은 조선 중기의 무신이다 [SEP]',
  'score': 0.8562734723091125,
  'token': 59906,
  'token_str': '조선'},
 {'sequence': '[CLS] 이순신은 청 중기의 무신이다 [SEP]',
  'score': 0.050685591995716095,
  'token': 9751,
  'token_str': '청'},
 {'sequence': '[CLS] 이순신은 전 중기의 무신이다 [SEP]',
  'score': 0.019383328035473824,
  'token': 9665,
  'token_str': '전'},
 {'sequence': '[CLS] 이순신은기 중기의 무신이다 [SEP]',
  'score': 0.007541158702224493,
  'token': 12310,
  'token_str': '##기'},
 {'sequence': '[CLS] 이순신은기의 중기의 무신이다 [SEP]',
  'score': 0.002962720114737749,
  'token': 46874,
  'token_str': '##기의'}]

모델의 출력 결과도 확인 가능

In [45]:
tokens_pt = tokenizer("이순신은 조선 중기의 무신이다.", return_tensors='pt')

for key, value in tokens_pt.items():
    print(key, value)

outputs = model(**tokens_pt) # input_ids, token_type_ids, attention_mask를 모두 입력하여 inference
last_hidden_state = outputs.last_hidden_state # 모든 hidden state를 담은 텐서
pooler_output = outputs.pooler_output # [CLS]에 대한 final hidden state

print('shape of last_hidden_state', last_hidden_state.shape)
print('shape of pooler_outputs', pooler_output.shape)

input_ids tensor([[   101,   9638, 119064,  25387,  10892,  59906,   9694,  46874,   9294,
          25387,  11925,    119,    102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
shape of last_hidden_state torch.Size([1, 13, 768])
shape of pooler_outputs torch.Size([1, 768])


In [49]:
# 기존 입력 임베딩 차원 수
print(model.get_input_embeddings())

# 새로 추가된 토큰 수
print('# of new tokens', added_token_num)

# 모델 입력 차원 수정
model.resize_token_embeddings(tokenizer.vocab_size + added_token_num)
print(model.get_input_embeddings()) # 차원이 늘어난 모습


Embedding(119547, 768, padding_idx=0)
# of new tokens 3
Embedding(119550, 768)


## `[CLS]` 토큰을 활용해 문장 간 유사도 측정 가능

In [51]:
sent1 = tokenizer('오늘 하루는 어떻게 보내셨나요?', return_tensors='pt')
sent2 = tokenizer('오늘은 어떤 하루를 보내셨나요?', return_tensors='pt')
sent3 = tokenizer('이순신은 조선 중기의 무신이다.', return_tensors='pt')
sent4 = tokenizer('머리카락이 켸쇽 나와서 우리가 청소 다 해야돼요', return_tensors='pt')

outputs = model(**sent1)
pooler_output1 = outputs.pooler_output

outputs = model(**sent2)
pooler_output2 = outputs.pooler_output

outputs = model(**sent3)
pooler_output3 = outputs.pooler_output

outputs = model(**sent4)
pooler_output4 = outputs.pooler_output



In [56]:
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
print(cos_sim(pooler_output1, pooler_output2)) # 문장 간 유사도가 높게 측정된 모습
print(cos_sim(pooler_output1, pooler_output3)) # 다른 맥락으로 유사도가 낮게 측정된 모습
print(cos_sim(pooler_output2, pooler_output3)) 

tensor([0.9896], grad_fn=<DivBackward0>)
tensor([0.5918], grad_fn=<DivBackward0>)
tensor([0.6075], grad_fn=<DivBackward0>)
